# Demo #9: Agentic RAG with Routing and Tools

## Objective
Build an autonomous agent that routes queries to appropriate data sources (vector DB, SQL, web search) and orchestrates multi-source information synthesis.

## Theoretical Background

### The Paradigm Shift: From Static Pipelines to Autonomous Agents

Traditional RAG systems follow a **static, predetermined workflow**: retrieve → augment → generate. This linear approach works well for simple question-answering but falls short when facing queries that require:
- Information from **multiple, heterogeneous data sources** (documents, databases, APIs)
- **Multi-step reasoning** where intermediate results inform subsequent retrieval decisions
- **Dynamic adaptation** based on what is (or isn't) found

**Agentic RAG** represents a paradigm shift to a **dynamic, autonomous process** orchestrated by an LLM. The LLM is elevated from a mere text generator to a **reasoning engine** that can:
- **Plan**: Decompose complex tasks into logical sub-tasks
- **Decide**: Choose which tools or data sources to use
- **Execute**: Call external tools via APIs (retrieval systems, databases, web search)
- **Observe**: Analyze the results and decide on the next action
- **Self-Correct**: Adapt strategy if initial attempts fail

This transforms the RAG system from a simple question-answering tool into an **autonomous problem-solver**.

### Core Agent Architectures

Several distinct agent-based architectural patterns have emerged:

#### 1. **Routing Agents**
In enterprise scenarios, knowledge is fragmented across multiple sources:
- **Vector database** for unstructured documents
- **SQL database** for transactional/structured data
- **Web search API** for current events
- **Knowledge graphs** for entity relationships

A **routing agent** acts as an **intelligent dispatcher**. It analyzes the user's query and determines the most appropriate data source or tool to use. This ability to intelligently query and synthesize information from diverse sources is what makes RAG truly viable in complex enterprise environments.

#### 2. **ReAct (Reason + Act) Agents**
The ReAct framework provides a powerful mechanism for **dynamic, multi-step reasoning**. An agent operating under this framework follows a **"Reason-Act-Observe" loop**:

1. **Reason**: Analyze the problem and formulate a plan
2. **Act**: Take an action (e.g., perform a retrieval, query a database)
3. **Observe**: Examine the outcome of that action
4. **Repeat**: Use new information to refine reasoning for the next step

This iterative process allows the agent to:
- Dynamically adjust its strategy
- Handle unexpected outcomes
- Perform self-correction

#### 3. **Plan-and-Execute Agents**
As an evolution of ReAct, these agents **separate planning and execution**:
1. First construct a complete, multi-step plan
2. Then execute the entire plan sequentially

This improves efficiency and reduces costs for tasks where the plan is unlikely to change based on intermediate results.

### The LLM as Meta-Reasoner

In agentic architectures, the LLM plays a **dual role**:
1. **Generator**: Still produces the final response
2. **Orchestrator**: More importantly, it becomes the control unit for the entire retrieval and reasoning process

It performs **meta-reasoning** about the task itself:
- "Is the information I have sufficient?"
- "Do I need to use a different tool?"
- "Should I rephrase my query?"
- "Can I answer this from my parametric knowledge, or do I need external data?"

This self-reflection and strategic planning represent a higher level of intelligence, positioning the LLM as the central "brain" for a complex, multi-component AI system.

### Multi-Source Data Synthesis Example

Consider the query: **"Compare the market sentiment for our new product with its Q1 sales figures."**

This cannot be answered by a single retrieval. An agentic system handles this by:
1. **Routing agent** identifies the query requires two types of information:
   - Sentiment (from web/news data)
   - Sales figures (from internal database)
2. Agent executes a search against a **web-focused tool** to retrieve articles and social media posts
3. Agent concurrently executes a query against a **SQL database tool** to retrieve Q1 sales data
4. LLM receives both sets of retrieved context to **synthesize a final answer** integrating both market sentiment and quantitative sales performance

## What We'll Demonstrate

In this demo, we'll build a **ReAct-based routing agent** that:
1. Has access to **three distinct tools**:
   - **Document Search Tool**: Queries a vector database of technical documentation
   - **Structured Data Tool**: Queries an in-memory "database" of structured product information
   - **Web Search Tool**: Simulated web search for current information
2. **Autonomously decides** which tool(s) to use based on the query
3. **Shows its reasoning process** (Reason-Act-Observe traces)
4. Can **synthesize information from multiple sources** in a single response

We'll compare this with a standard, single-source RAG system to highlight the advantages of agentic architecture.

---

**References:**
- *What is Agentic RAG?* | IBM (Reference 57)
- *GraphRAG and Agentic Architecture* - Neo4j (Reference 55)
- Curriculum: "The Autonomous Frontier: Agentic RAG Architectures"

## 1. Environment Setup and Dependencies

In [1]:
import os
from pathlib import Path
import json
from typing import List, Dict, Any

# LlamaIndex core
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Settings
)

# Azure OpenAI
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

# Agent and Tools
from llama_index.core.tools import QueryEngineTool, FunctionTool
from llama_index.core.agent import ReActAgent

print("✓ All imports successful")

✓ All imports successful


## 2. Azure OpenAI Configuration

In [3]:
# Azure OpenAI Configuration
llm = AzureOpenAI(
    model="gpt-4",
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version="2024-02-15-preview",
    temperature=0.1  # Low temperature for more deterministic reasoning
)

embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME", "text-embedding-ada-002"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version="2024-02-15-preview"
)

# Configure global settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512
Settings.chunk_overlap = 50

print("✓ Azure OpenAI configured")
print(f"  LLM: {llm.model}")
print(f"  Embedding: text-embedding-ada-002")

✓ Azure OpenAI configured
  LLM: gpt-4
  Embedding: text-embedding-ada-002


## 3. Multi-Source Data Setup

We'll create three distinct data sources to simulate a real enterprise environment:

### Source 1: Vector Database (Unstructured Documents)
Technical documentation stored in markdown files

In [4]:
# Load technical documentation
data_dir = Path("./data/tech_docs")

print(f"Loading documents from: {data_dir}")
documents = SimpleDirectoryReader(
    input_dir=str(data_dir),
    required_exts=[".md"]
).load_data()

print(f"✓ Loaded {len(documents)} documents")
for doc in documents:
    filename = Path(doc.metadata['file_path']).name
    print(f"  - {filename}")

Loading documents from: data/tech_docs
✓ Loaded 6 documents
  - bert_model.md
  - docker_containers.md
  - embeddings_ml.md
  - gpt4_model.md
  - rest_api.md
  - transformer_architecture.md


In [ ]:
# Create vector index
print("Building vector index...")
vector_index = VectorStoreIndex.from_documents(
    documents,
    show_progress=True
)

# Create query engine
vector_query_engine = vector_index.as_query_engine(
    similarity_top_k=3,
    response_mode="compact"
)

print("✓ Vector index created")

### Source 2: Structured Database (Simulated SQL)
Product and sales data in a structured format

In [5]:
# Simulated structured database
# In a real system, this would be a SQL database, but we'll use an in-memory dict
structured_database = {
    "products": [
        {
            "id": "P001",
            "name": "CloudFlow API Gateway",
            "category": "API Management",
            "price": 299.99,
            "release_date": "2024-01-15"
        },
        {
            "id": "P002",
            "name": "DockerPro Container Suite",
            "category": "DevOps",
            "price": 499.99,
            "release_date": "2023-11-20"
        },
        {
            "id": "P003",
            "name": "TransformerAI Model Kit",
            "category": "Machine Learning",
            "price": 1299.99,
            "release_date": "2024-03-10"
        }
    ],
    "sales_q1_2024": [
        {"product_id": "P001", "units_sold": 1520, "revenue": 455848.00},
        {"product_id": "P002", "units_sold": 890, "revenue": 444991.00},
        {"product_id": "P003", "units_sold": 340, "revenue": 441996.60}
    ],
    "sales_q2_2024": [
        {"product_id": "P001", "units_sold": 1680, "revenue": 503832.00},
        {"product_id": "P002", "units_sold": 1050, "revenue": 524989.50},
        {"product_id": "P003", "units_sold": 420, "revenue": 545995.80}
    ]
}

def query_structured_data(query: str) -> str:
    """
    Queries the structured database based on natural language input.
    This simulates a SQL query interface.
    
    Args:
        query: Natural language query about products or sales
    
    Returns:
        JSON string with relevant structured data
    """
    query_lower = query.lower()
    
    results = {}
    
    # Simple keyword-based routing (in real systems, use NL-to-SQL)
    if any(word in query_lower for word in ['product', 'price', 'category', 'release']):
        results['products'] = structured_database['products']
    
    if any(word in query_lower for word in ['sales', 'revenue', 'sold', 'q1', 'q2', 'quarter']):
        if 'q1' in query_lower or 'first quarter' in query_lower:
            results['sales_q1_2024'] = structured_database['sales_q1_2024']
        elif 'q2' in query_lower or 'second quarter' in query_lower:
            results['sales_q2_2024'] = structured_database['sales_q2_2024']
        else:
            # Return all sales data if no specific quarter mentioned
            results['sales_q1_2024'] = structured_database['sales_q1_2024']
            results['sales_q2_2024'] = structured_database['sales_q2_2024']
    
    # If no matches, return all data
    if not results:
        results = structured_database
    
    return json.dumps(results, indent=2)

# Test the function
print("Testing structured data query:")
test_result = query_structured_data("What were the Q1 sales?")
print(test_result[:200] + "...")
print("\n✓ Structured database configured")

Testing structured data query:
{
  "sales_q1_2024": [
    {
      "product_id": "P001",
      "units_sold": 1520,
      "revenue": 455848.0
    },
    {
      "product_id": "P002",
      "units_sold": 890,
      "revenue": 444991.0...

✓ Structured database configured


### Source 3: Web Search (Simulated)
A simulated web search tool for "current" information not in our knowledge base

In [6]:
def web_search(query: str) -> str:
    """
    Simulates a web search API (like Bing or Google Search).
    In production, this would call an actual search API.
    
    Args:
        query: Search query string
    
    Returns:
        Simulated search results as a formatted string
    """
    # Simulated search results for demonstration
    search_results = {
        "ai trends": [
            {
                "title": "Top AI Trends in 2024 - TechCrunch",
                "snippet": "The AI landscape continues to evolve rapidly with multimodal models, improved reasoning capabilities, and more efficient training methods leading the charge.",
                "date": "2024-10-10"
            },
            {
                "title": "Enterprise AI Adoption Reaches New Heights",
                "snippet": "Recent surveys show 78% of enterprises are now deploying AI in production, with RAG-based systems being the most common architecture.",
                "date": "2024-10-08"
            }
        ],
        "market sentiment": [
            {
                "title": "AI Market Sentiment Analysis - October 2024",
                "snippet": "Market analysts report overwhelmingly positive sentiment towards AI infrastructure products, particularly those focused on enterprise deployment.",
                "date": "2024-10-12"
            }
        ],
        "gpt-5": [
            {
                "title": "OpenAI Announces GPT-5 Development",
                "snippet": "OpenAI has confirmed that GPT-5 is in development, promising significant improvements in reasoning and reduced hallucination rates.",
                "date": "2024-10-15"
            }
        ]
    }
    
    query_lower = query.lower()
    
    # Find matching results
    relevant_results = []
    for keyword, results in search_results.items():
        if keyword in query_lower:
            relevant_results.extend(results)
    
    # Default results if no match
    if not relevant_results:
        return f"Web search for '{query}': No recent results found. This information may not be available via web search."
    
    # Format results
    formatted = f"Web search results for '{query}':\n\n"
    for i, result in enumerate(relevant_results, 1):
        formatted += f"{i}. {result['title']} ({result['date']})\n"
        formatted += f"   {result['snippet']}\n\n"
    
    return formatted

# Test the function
print("Testing web search:")
test_result = web_search("What are the latest AI trends?")
print(test_result)
print("✓ Web search tool configured")

Testing web search:
Web search results for 'What are the latest AI trends?':

1. Top AI Trends in 2024 - TechCrunch (2024-10-10)
   The AI landscape continues to evolve rapidly with multimodal models, improved reasoning capabilities, and more efficient training methods leading the charge.

2. Enterprise AI Adoption Reaches New Heights (2024-10-08)
   Recent surveys show 78% of enterprises are now deploying AI in production, with RAG-based systems being the most common architecture.


✓ Web search tool configured


## 4. Tool Definitions for the Agent

Now we'll wrap each data source in a **Tool** that the agent can use. Each tool has:
- A **name**: Identifier for the tool
- A **description**: Critical! This tells the agent *when* to use this tool
- A **function**: The actual callable that performs the action

In [7]:
# Tool 1: Document Search (Vector Database)
document_search_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    name="document_search",
    description=(
        "Use this tool to search technical documentation about software technologies, "
        "APIs, machine learning models, and software architectures. "
        "This tool is best for questions about 'how does X work?', 'what is X?', "
        "or 'explain the architecture of X'."
    )
)

# Tool 2: Structured Data Query (Simulated SQL)
structured_data_tool = FunctionTool.from_defaults(
    fn=query_structured_data,
    name="database_query",
    description=(
        "Use this tool to query structured business data including product information "
        "(names, prices, categories, release dates) and sales data (revenue, units sold). "
        "This tool is best for questions about 'what products do we have?', "
        "'what were the Q1/Q2 sales?', or 'how much revenue did product X generate?'."
    )
)

# Tool 3: Web Search (Simulated)
web_search_tool = FunctionTool.from_defaults(
    fn=web_search,
    name="web_search",
    description=(
        "Use this tool to search for current information, recent news, market trends, "
        "or information that is not in the internal documentation or database. "
        "This tool is best for questions about 'latest trends', 'current market sentiment', "
        "'recent developments', or 'what's new in X?'."
    )
)

# Collect all tools
agent_tools = [document_search_tool, structured_data_tool, web_search_tool]

print("✓ Created 3 tools for the agent:")
for tool in agent_tools:
    print(f"  - {tool.metadata.name}: {tool.metadata.description[:80]}...")

NameError: name 'vector_query_engine' is not defined

## 5. Create the ReAct Agent

The **ReActAgent** implements the Reason-Act-Observe loop. It will:
1. Reason about which tool(s) to use
2. Act by calling the selected tool(s)
3. Observe the results
4. Repeat if needed or synthesize a final answer

In [ ]:
# Create the ReAct agent
agent = ReActAgent.from_tools(
    agent_tools,
    llm=llm,
    verbose=True,  # Show reasoning traces
    max_iterations=10  # Maximum reasoning steps to prevent infinite loops
)

print("✓ ReAct Agent created")
print(f"  Available tools: {len(agent_tools)}")
print(f"  Max iterations: 10")
print(f"  Verbose mode: Enabled (will show reasoning traces)")

## 6. Test Scenarios: Single-Source Queries

Let's start with queries that require only one data source. We'll observe how the agent **autonomously decides** which tool to use.

### Scenario 1: Document Search Query
Query about technical concepts (should use `document_search`)

In [ ]:
print("="*80)
print("SCENARIO 1: Technical Documentation Query")
print("="*80)

query_1 = "What is the transformer architecture and how does it work?"
print(f"\nQuery: {query_1}\n")

response_1 = agent.chat(query_1)

print("\n" + "="*80)
print("FINAL RESPONSE:")
print("="*80)
print(response_1)

**Observe the agent's reasoning trace above:**
- The agent should recognize this is a technical question
- It should select the `document_search` tool
- It should provide an answer based on the retrieved documentation

### Scenario 2: Structured Data Query
Query about business data (should use `database_query`)

In [ ]:
print("="*80)
print("SCENARIO 2: Structured Database Query")
print("="*80)

query_2 = "What were the Q2 2024 sales figures for all products?"
print(f"\nQuery: {query_2}\n")

response_2 = agent.chat(query_2)

print("\n" + "="*80)
print("FINAL RESPONSE:")
print("="*80)
print(response_2)

**Observe the agent's reasoning:**
- The agent should recognize this requires structured sales data
- It should select the `database_query` tool
- It should provide formatted sales figures from the database

### Scenario 3: Web Search Query
Query about current events (should use `web_search`)

In [ ]:
print("="*80)
print("SCENARIO 3: Current Information / Web Search Query")
print("="*80)

query_3 = "What are the latest trends in AI for 2024?"
print(f"\nQuery: {query_3}\n")

response_3 = agent.chat(query_3)

print("\n" + "="*80)
print("FINAL RESPONSE:")
print("="*80)
print(response_3)

**Observe the agent's reasoning:**
- The agent should recognize this requires current/recent information
- It should select the `web_search` tool
- It should synthesize the simulated web search results

## 7. Advanced Scenario: Multi-Source Query

Now the real power: a query that **requires information from multiple sources**.

In [ ]:
print("="*80)
print("SCENARIO 4: Multi-Source Query (The Agentic RAG Advantage)")
print("="*80)

query_4 = (
    "Compare our TransformerAI Model Kit product with current AI market trends. "
    "Include pricing information and recent sales performance."
)
print(f"\nQuery: {query_4}\n")
print("This query requires:")
print("  1. Product information (database_query)")
print("  2. Technical details about transformers (document_search)")
print("  3. Market trends (web_search)")
print("\nLet's see how the agent handles this...\n")

response_4 = agent.chat(query_4)

print("\n" + "="*80)
print("FINAL RESPONSE:")
print("="*80)
print(response_4)

**Observe the multi-step reasoning:**
- The agent should identify that this query requires multiple pieces of information
- It should make multiple tool calls:
  - `database_query` for product pricing and sales data
  - `web_search` for market trends
  - Possibly `document_search` for technical context on transformers
- It should **synthesize** all this information into a coherent response

This is the quintessential **Agentic RAG** capability: autonomous orchestration of multiple data sources.

## 8. Comparative Analysis: Agentic vs. Standard RAG

Let's demonstrate what happens when we try to answer the multi-source query with a **standard, single-source RAG system**.

In [ ]:
print("="*80)
print("COMPARISON: Standard RAG (Document-Only) vs. Agentic RAG")
print("="*80)

print("\n--- STANDARD RAG (Vector Search Only) ---\n")
standard_rag_response = vector_query_engine.query(query_4)
print(f"Query: {query_4}\n")
print(f"Response: {standard_rag_response}\n")

print("\n--- AGENTIC RAG (Multi-Source) ---\n")
print(f"Query: {query_4}\n")
print(f"Response: {response_4}\n")

## 9. Analysis and Key Takeaways

In [ ]:
from IPython.display import display, Markdown

analysis = """
## Comparative Analysis

### Standard RAG Limitations

The standard vector-only RAG system:
- ✗ **Cannot access structured data** (pricing, sales figures)
- ✗ **Cannot access current information** (market trends from web)
- ✗ **Limited to one data source** (technical documentation only)
- ✗ **Inflexible**: Always uses the same retrieval strategy regardless of query type

For the multi-source query, it likely:
- Provided technical information about transformers (what it *can* retrieve)
- Could not provide pricing or sales data (not in documents)
- Could not provide market trends (not in its knowledge base)
- **Incomplete answer** due to architectural limitations

### Agentic RAG Advantages

The agentic system:
- ✓ **Multi-source capability**: Can query vector DB, structured DB, and web search
- ✓ **Autonomous routing**: Intelligently selects the right tool(s) for each query
- ✓ **Dynamic workflow**: Can adjust strategy based on intermediate results
- ✓ **Information synthesis**: Combines data from multiple sources into coherent answers
- ✓ **Transparent reasoning**: Shows its decision-making process (Reason-Act-Observe traces)
- ✓ **Self-correcting**: Can try different approaches if initial attempts fail

For the multi-source query, it:
- Identified the need for product data → called `database_query`
- Identified the need for market trends → called `web_search`
- Possibly identified need for technical context → called `document_search`
- **Synthesized all sources** into a comprehensive, complete answer

## The Paradigm Shift

| Aspect | Traditional RAG | Agentic RAG |
|--------|----------------|-------------|
| **Workflow** | Static: retrieve → generate | Dynamic: reason → act → observe → repeat |
| **Data Sources** | Single source (typically vector DB) | Multiple heterogeneous sources |
| **Decision Making** | Fixed pipeline | LLM-orchestrated, adaptive |
| **LLM Role** | Generator only | Generator + Orchestrator + Meta-reasoner |
| **Complexity** | Simple, linear | Complex, iterative |
| **Best For** | Straightforward Q&A over documents | Multi-source synthesis, complex reasoning tasks |
| **Enterprise Viability** | Limited to document search use cases | Viable for complex, real-world scenarios |

## When to Use Agentic RAG

**Choose Agentic RAG when:**
1. Queries require information from **multiple data sources** (documents, databases, APIs)
2. You need **dynamic routing** based on query characteristics
3. Tasks involve **multi-step reasoning** where intermediate results inform next steps
4. You need **transparency** in the system's decision-making process
5. The domain requires **synthesis** of structured + unstructured + external data

**Stick with traditional RAG when:**
1. All information is in a **single, homogeneous source**
2. Queries are **straightforward** and don't require multi-step reasoning
3. **Latency** and **cost** are critical (agents make multiple LLM calls)
4. The use case is **document search only**

## The LLM as Meta-Reasoner

The key insight of Agentic RAG is that the LLM performs **meta-reasoning**:
- "What information do I need?"
- "Where can I find it?"
- "Is what I found sufficient?"
- "Do I need to search again with a different strategy?"

This elevates the LLM from a **text generator** to the **central intelligence** of a complex system—the "brain" that coordinates multiple specialized tools to solve problems that are intractable for simpler architectures.

## Production Considerations

When deploying Agentic RAG in production:

1. **Cost Management**: Agents make multiple LLM calls. Monitor token usage carefully.
2. **Latency**: Multi-step reasoning takes time. Consider async execution and user experience.
3. **Reliability**: Implement max iteration limits and fallback strategies to prevent infinite loops.
4. **Tool Descriptions**: The quality of tool descriptions is critical—they guide the agent's routing decisions.
5. **Observability**: Log all reasoning traces for debugging and system improvement.
6. **Security**: Validate tool calls and implement access controls for sensitive data sources.
"""

display(Markdown(analysis))

## 10. Conclusion

### What We Demonstrated

In this demo, we successfully built and tested an **Agentic RAG system** with:

1. **Three heterogeneous data sources**:
   - Vector database for technical documentation
   - Structured database for product/sales data
   - Web search for current information

2. **ReAct agent architecture** with:
   - Autonomous tool selection and routing
   - Transparent Reason-Act-Observe reasoning traces
   - Multi-source information synthesis

3. **Comprehensive testing** showing:
   - Correct single-source routing for simple queries
   - Multi-source orchestration for complex queries
   - Clear advantages over standard, single-source RAG

### Architectural Significance

Agentic RAG represents a fundamental evolution:
- **From static pipelines to dynamic orchestration**
- **From single-source to multi-source synthesis**
- **From passive generators to active meta-reasoners**

This architecture makes RAG viable for **real-world enterprise scenarios** where knowledge is fragmented across multiple systems and queries require intelligent routing and multi-step reasoning.

### Integration with Previous Concepts

Agentic RAG builds on and extends the techniques from previous demos:
- **Query enhancement** (HyDE, multi-query) can be tools the agent selects
- **Hybrid search** can be one of multiple retrieval strategies
- **Re-ranking** can be applied to results from any tool
- **Corrective RAG** and **Self-RAG** are natural fits for agent-based self-correction

The agent becomes the **orchestration layer** that intelligently applies these techniques based on the specific characteristics of each query.

### Next Steps

To extend this demo:
1. **Add more tools**: Image search, code execution, real-time data feeds
2. **Implement plan-and-execute**: Pre-plan multi-step queries for efficiency
3. **Add memory**: Allow the agent to remember conversation context
4. **Integrate GraphRAG**: Add graph traversal as another tool
5. **Production hardening**: Add error handling, retry logic, and observability

---

**References:**
- Curriculum: "The Autonomous Frontier: Agentic RAG Architectures"
- *What is Agentic RAG?* | IBM (Reference 57)
- *GraphRAG and Agentic Architecture* - Neo4j (Reference 55)